In [1]:
## Import Packages
import numpy as np
import scipy.stats as stats
import os
import pickle
import scipy.io as scio
from ncdump_python3 import ncdump
import pickle
import Area_Avg
from scipy.interpolate import griddata
from netCDF4 import Dataset
import warnings
warnings.filterwarnings('ignore')
np.set_printoptions(suppress=True)

In [2]:
#Load in Dimensions
LLL = Dataset('LatLon.nc')
WACCM4_Lat = np.squeeze(LLL.variables['lat'])
WACCM4_Lon = np.squeeze(LLL.variables['lon'])
LLL.close()

Months = np.arange(0,12)
FP_Ensemble_Members = np.arange(0,5)

In [3]:
#load climate change responses
RCP45_dFLUT = pickle.load(open(\
    "Future Projection CC Responses/RCP45_dFLUT.pickle","rb"),encoding='latin1')
RCP45_dFSNT = pickle.load(open(\
    "Future Projection CC Responses/RCP45_dFSNT.pickle","rb"),encoding='latin1')
RCP45_dST = pickle.load(open(\
    "Future Projection CC Responses/RCP45_dST.pickle","rb"),encoding='latin1')

RCP45_dOHC_Annual_AA = pickle.load(open(\
"ENERGY BUDGET/Ocean Heat Content/RCP45/RCP45_OCN_EnergyB_AA.pickle","rb"))
RCP45_dOHC_SON_AA = pickle.load(open(\
"ENERGY BUDGET/Ocean Heat Content/RCP45/Seasonal Breakdown/RCP45_OCN_EnergyB_SON_AA.pickle","rb"))
RCP45_dOHC_MAM_AA = pickle.load(open(\
"ENERGY BUDGET/Ocean Heat Content/RCP45/Seasonal Breakdown/RCP45_OCN_EnergyB_MAM_AA.pickle","rb"))
RCP45_dOHC_JJA_AA = pickle.load(open(\
"ENERGY BUDGET/Ocean Heat Content/RCP45/Seasonal Breakdown/RCP45_OCN_EnergyB_JJA_AA.pickle","rb"))
RCP45_dOHC_DJF_AA = pickle.load(open(\
"ENERGY BUDGET/Ocean Heat Content/RCP45/Seasonal Breakdown/RCP45_OCN_EnergyB_DJF_AA.pickle","rb"))

RCP45_dOHC_Annual_GA = pickle.load(open(\
"ENERGY BUDGET/Ocean Heat Content/RCP45/RCP45_OCN_EnergyB_GA.pickle","rb"))
RCP45_dOHC_SON_GA = pickle.load(open(\
"ENERGY BUDGET/Ocean Heat Content/RCP45/Seasonal Breakdown/RCP45_OCN_EnergyB_SON_GA.pickle","rb"))
RCP45_dOHC_MAM_GA = pickle.load(open(\
"ENERGY BUDGET/Ocean Heat Content/RCP45/Seasonal Breakdown/RCP45_OCN_EnergyB_MAM_GA.pickle","rb"))
RCP45_dOHC_JJA_GA = pickle.load(open(\
"ENERGY BUDGET/Ocean Heat Content/RCP45/Seasonal Breakdown/RCP45_OCN_EnergyB_JJA_GA.pickle","rb"))
RCP45_dOHC_DJF_GA = pickle.load(open(\
"ENERGY BUDGET/Ocean Heat Content/RCP45/Seasonal Breakdown/RCP45_OCN_EnergyB_DJF_GA.pickle","rb"))

In [4]:
#Split fluxes into seasons
RCP45_dFLUT_SON = RCP45_dFLUT[:,8:11,:,:]
RCP45_dFLUT_JJA = RCP45_dFLUT[:,5:8,:,:]
RCP45_dFLUT_MAM = RCP45_dFLUT[:,2:5,:,:]
RCP45_dFLUT_DJF_1 = RCP45_dFLUT[:,0:2,:,:]
RCP45_dFLUT_DJF_2 = np.expand_dims(RCP45_dFLUT[:,11,:,:],axis=1)
RCP45_dFLUT_DJF = np.concatenate((RCP45_dFLUT_DJF_1,\
                                         RCP45_dFLUT_DJF_2),axis=1)

RCP45_dFSNT_SON = RCP45_dFSNT[:,8:11,:,:]
RCP45_dFSNT_JJA = RCP45_dFSNT[:,5:8,:,:]
RCP45_dFSNT_MAM = RCP45_dFSNT[:,2:5,:,:]
RCP45_dFSNT_DJF_1 = RCP45_dFSNT[:,0:2,:,:]
RCP45_dFSNT_DJF_2 = np.expand_dims(RCP45_dFSNT[:,11,:,:],axis=1)
RCP45_dFSNT_DJF = np.concatenate((RCP45_dFSNT_DJF_1,\
                                         RCP45_dFSNT_DJF_2),axis=1)

RCP45_dST_SON = RCP45_dST[:,8:11,:,:]
RCP45_dST_JJA = RCP45_dST[:,5:8,:,:]
RCP45_dST_MAM = RCP45_dST[:,2:5,:,:]
RCP45_dST_DJF_1 = RCP45_dST[:,0:2,:,:]
RCP45_dST_DJF_2 = np.expand_dims(RCP45_dST[:,11,:,:],axis=1)
RCP45_dST_DJF = np.concatenate((RCP45_dST_DJF_1,\
                                         RCP45_dST_DJF_2),axis=1)

In [15]:
RCP45_dNET_TOA = RCP45_dFSNT-RCP45_dFLUT
RCP45_dNET_TOA_SON = RCP45_dFSNT_SON-RCP45_dFLUT_SON
RCP45_dNET_TOA_JJA = RCP45_dFSNT_JJA-RCP45_dFLUT_JJA
RCP45_dNET_TOA_DJF = RCP45_dFSNT_DJF-RCP45_dFLUT_DJF
RCP45_dNET_TOA_MAM = RCP45_dFSNT_MAM-RCP45_dFLUT_MAM

RCP45_dNET_TOA_EnergyB_AA = np.zeros([FP_Ensemble_Members.size])
RCP45_dNET_TOA_EnergyB_SON_AA = np.zeros([FP_Ensemble_Members.size])
RCP45_dNET_TOA_EnergyB_JJA_AA = np.zeros([FP_Ensemble_Members.size])
RCP45_dNET_TOA_EnergyB_DJF_AA = np.zeros([FP_Ensemble_Members.size])
RCP45_dNET_TOA_EnergyB_MAM_AA = np.zeros([FP_Ensemble_Members.size])

RCP45_dNET_TOA_EnergyB_GA = np.zeros([FP_Ensemble_Members.size])
RCP45_dNET_TOA_EnergyB_SON_GA = np.zeros([FP_Ensemble_Members.size])
RCP45_dNET_TOA_EnergyB_JJA_GA = np.zeros([FP_Ensemble_Members.size])
RCP45_dNET_TOA_EnergyB_DJF_GA = np.zeros([FP_Ensemble_Members.size])
RCP45_dNET_TOA_EnergyB_MAM_GA = np.zeros([FP_Ensemble_Members.size])

for i in range(FP_Ensemble_Members.size):
    RCP45_dNET_TOA_EnergyB_AA[i] = np.mean(Area_Avg.LatLonavg_Time(\
        RCP45_dNET_TOA[i,:,80:,:],WACCM4_Lat[80:],WACCM4_Lon),axis=0)
    
    RCP45_dNET_TOA_EnergyB_SON_AA[i] = np.mean(Area_Avg.LatLonavg_Time(\
        RCP45_dNET_TOA_SON[i,:,80:,:],WACCM4_Lat[80:],WACCM4_Lon),axis=0)
    
    RCP45_dNET_TOA_EnergyB_JJA_AA[i] = np.mean(Area_Avg.LatLonavg_Time(\
        RCP45_dNET_TOA_JJA[i,:,80:,:],WACCM4_Lat[80:],WACCM4_Lon),axis=0)
    
    RCP45_dNET_TOA_EnergyB_DJF_AA[i] = np.mean(Area_Avg.LatLonavg_Time(\
        RCP45_dNET_TOA_DJF[i,:,80:,:],WACCM4_Lat[80:],WACCM4_Lon),axis=0)
    
    RCP45_dNET_TOA_EnergyB_MAM_AA[i] = np.mean(Area_Avg.LatLonavg_Time(\
        RCP45_dNET_TOA_MAM[i,:,80:,:],WACCM4_Lat[80:],WACCM4_Lon),axis=0)
    
    RCP45_dNET_TOA_EnergyB_GA[i] = np.mean(Area_Avg.LatLonavg_Time(\
        RCP45_dNET_TOA[i,:,:,:],WACCM4_Lat[:],WACCM4_Lon),axis=0)
    
    RCP45_dNET_TOA_EnergyB_SON_GA[i] = np.mean(Area_Avg.LatLonavg_Time(\
        RCP45_dNET_TOA_SON[i,:,:,:],WACCM4_Lat[:],WACCM4_Lon),axis=0)
    
    RCP45_dNET_TOA_EnergyB_JJA_GA[i] = np.mean(Area_Avg.LatLonavg_Time(\
        RCP45_dNET_TOA_JJA[i,:,:,:],WACCM4_Lat[:],WACCM4_Lon),axis=0)
    
    RCP45_dNET_TOA_EnergyB_DJF_GA[i] = np.mean(Area_Avg.LatLonavg_Time(\
        RCP45_dNET_TOA_DJF[i,:,:,:],WACCM4_Lat[:],WACCM4_Lon),axis=0)
    
    RCP45_dNET_TOA_EnergyB_MAM_GA[i] = np.mean(Area_Avg.LatLonavg_Time(\
        RCP45_dNET_TOA_MAM[i,:,:,:],WACCM4_Lat[:],WACCM4_Lon),axis=0)

In [16]:
RCP45_DYN_EnergyB_Annual_AA = -(RCP45_dNET_TOA_EnergyB_AA-RCP45_dOHC_Annual_AA)
RCP45_DYN_EnergyB_SON_AA = -(RCP45_dNET_TOA_EnergyB_SON_AA-RCP45_dOHC_SON_AA)
RCP45_DYN_EnergyB_DJF_AA = -(RCP45_dNET_TOA_EnergyB_DJF_AA-RCP45_dOHC_DJF_AA)
RCP45_DYN_EnergyB_JJA_AA = -(RCP45_dNET_TOA_EnergyB_JJA_AA-RCP45_dOHC_JJA_AA)
RCP45_DYN_EnergyB_MAM_AA = -(RCP45_dNET_TOA_EnergyB_MAM_AA-RCP45_dOHC_MAM_AA)

In [17]:
RCP45_DYN_EnergyB_Annual_GA = -(RCP45_dNET_TOA_EnergyB_GA-RCP45_dOHC_Annual_GA)
RCP45_DYN_EnergyB_SON_GA = -(RCP45_dNET_TOA_EnergyB_SON_GA-RCP45_dOHC_SON_GA)
RCP45_DYN_EnergyB_DJF_GA = -(RCP45_dNET_TOA_EnergyB_DJF_GA-RCP45_dOHC_DJF_GA)
RCP45_DYN_EnergyB_JJA_GA = -(RCP45_dNET_TOA_EnergyB_JJA_GA-RCP45_dOHC_JJA_GA)
RCP45_DYN_EnergyB_MAM_GA = -(RCP45_dNET_TOA_EnergyB_MAM_GA-RCP45_dOHC_MAM_GA)

In [18]:
#Area average surface temperature change over 60-90N and globally
RCP45_dST_AA = np.zeros([FP_Ensemble_Members.size])
RCP45_dST_SON_AA = np.zeros([FP_Ensemble_Members.size])
RCP45_dST_DJF_AA = np.zeros([FP_Ensemble_Members.size])
RCP45_dST_JJA_AA = np.zeros([FP_Ensemble_Members.size])
RCP45_dST_MAM_AA = np.zeros([FP_Ensemble_Members.size])


for i in range(FP_Ensemble_Members.size):    
    RCP45_dST_SON_AA[i] = np.ma.mean(Area_Avg.LatLonavg_Time(\
    RCP45_dST_SON[i,:,80:,:],WACCM4_Lat[80:],WACCM4_Lon[:]),axis=0)
    
    RCP45_dST_DJF_AA[i] = np.ma.mean(Area_Avg.LatLonavg_Time(\
    RCP45_dST_DJF[i,:,80:,:],WACCM4_Lat[80:],WACCM4_Lon[:]),axis=0)
    
    RCP45_dST_JJA_AA[i] = np.ma.mean(Area_Avg.LatLonavg_Time(\
    RCP45_dST_JJA[i,:,80:,:],WACCM4_Lat[80:],WACCM4_Lon[:]),axis=0)
    
    RCP45_dST_MAM_AA[i] = np.ma.mean(Area_Avg.LatLonavg_Time(\
    RCP45_dST_MAM[i,:,80:,:],WACCM4_Lat[80:],WACCM4_Lon[:]),axis=0)
    
    RCP45_dST_AA[i] = np.ma.mean(Area_Avg.LatLonavg_Time(\
    RCP45_dST[i,:,80:,:],WACCM4_Lat[80:],WACCM4_Lon[:]),axis=0)

In [19]:
RCP45_DYN_Feedback_Annual_AA = RCP45_DYN_EnergyB_Annual_AA/RCP45_dST_AA
RCP45_DYN_Feedback_SON_AA = RCP45_DYN_EnergyB_SON_AA/RCP45_dST_SON_AA
RCP45_DYN_Feedback_DJF_AA = RCP45_DYN_EnergyB_DJF_AA/RCP45_dST_DJF_AA
RCP45_DYN_Feedback_JJA_AA = RCP45_DYN_EnergyB_JJA_AA/RCP45_dST_JJA_AA
RCP45_DYN_Feedback_MAM_AA = RCP45_DYN_EnergyB_MAM_AA/RCP45_dST_MAM_AA

In [20]:
RCP45_DYN_EnergyB_Annual_AA_file = open("RCP45_DYN_EnergyB_AA.pickle","wb")
pickle.dump(RCP45_DYN_EnergyB_Annual_AA,RCP45_DYN_EnergyB_Annual_AA_file)
RCP45_DYN_EnergyB_Annual_AA_file.close()

RCP45_DYN_EnergyB_SON_AA_file = open("RCP45_DYN_EnergyB_SON_AA.pickle","wb")
pickle.dump(RCP45_DYN_EnergyB_SON_AA,RCP45_DYN_EnergyB_SON_AA_file)
RCP45_DYN_EnergyB_SON_AA_file.close()

RCP45_DYN_EnergyB_JJA_AA_file = open("RCP45_DYN_EnergyB_JJA_AA.pickle","wb")
pickle.dump(RCP45_DYN_EnergyB_JJA_AA,RCP45_DYN_EnergyB_JJA_AA_file)
RCP45_DYN_EnergyB_JJA_AA_file.close()

RCP45_DYN_EnergyB_MAM_AA_file = open("RCP45_DYN_EnergyB_MAM_AA.pickle","wb")
pickle.dump(RCP45_DYN_EnergyB_MAM_AA,RCP45_DYN_EnergyB_MAM_AA_file)
RCP45_DYN_EnergyB_MAM_AA_file.close()

RCP45_DYN_EnergyB_DJF_AA_file = open("RCP45_DYN_EnergyB_DJF_AA.pickle","wb")
pickle.dump(RCP45_DYN_EnergyB_DJF_AA,RCP45_DYN_EnergyB_DJF_AA_file)
RCP45_DYN_EnergyB_DJF_AA_file.close()

RCP45_DYN_Feedback_Annual_AA_file = open("RCP45_DYN_Feedback_AA.pickle","wb")
pickle.dump(RCP45_DYN_Feedback_Annual_AA,RCP45_DYN_Feedback_Annual_AA_file)
RCP45_DYN_Feedback_Annual_AA_file.close()

RCP45_DYN_Feedback_SON_AA_file = open("RCP45_DYN_Feedback_SON_AA.pickle","wb")
pickle.dump(RCP45_DYN_Feedback_SON_AA,RCP45_DYN_Feedback_SON_AA_file)
RCP45_DYN_Feedback_SON_AA_file.close()

RCP45_DYN_Feedback_JJA_AA_file = open("RCP45_DYN_Feedback_JJA_AA.pickle","wb")
pickle.dump(RCP45_DYN_Feedback_JJA_AA,RCP45_DYN_Feedback_JJA_AA_file)
RCP45_DYN_Feedback_JJA_AA_file.close()

RCP45_DYN_Feedback_MAM_AA_file = open("RCP45_DYN_Feedback_MAM_AA.pickle","wb")
pickle.dump(RCP45_DYN_Feedback_MAM_AA,RCP45_DYN_Feedback_MAM_AA_file)
RCP45_DYN_Feedback_MAM_AA_file.close()

RCP45_DYN_Feedback_DJF_AA_file = open("RCP45_DYN_Feedback_DJF_AA.pickle","wb")
pickle.dump(RCP45_DYN_Feedback_DJF_AA,RCP45_DYN_Feedback_DJF_AA_file)
RCP45_DYN_Feedback_DJF_AA_file.close()

In [21]:
RCP45_DYN_EnergyB_Annual_GA_file = open("RCP45_DYN_EnergyB_GA.pickle","wb")
pickle.dump(RCP45_DYN_EnergyB_Annual_GA,RCP45_DYN_EnergyB_Annual_GA_file)
RCP45_DYN_EnergyB_Annual_GA_file.close()

RCP45_DYN_EnergyB_SON_GA_file = open("RCP45_DYN_EnergyB_SON_GA.pickle","wb")
pickle.dump(RCP45_DYN_EnergyB_SON_GA,RCP45_DYN_EnergyB_SON_GA_file)
RCP45_DYN_EnergyB_SON_GA_file.close()

RCP45_DYN_EnergyB_JJA_GA_file = open("RCP45_DYN_EnergyB_JJA_GA.pickle","wb")
pickle.dump(RCP45_DYN_EnergyB_JJA_GA,RCP45_DYN_EnergyB_JJA_GA_file)
RCP45_DYN_EnergyB_JJA_GA_file.close()

RCP45_DYN_EnergyB_MAM_GA_file = open("RCP45_DYN_EnergyB_MAM_GA.pickle","wb")
pickle.dump(RCP45_DYN_EnergyB_MAM_GA,RCP45_DYN_EnergyB_MAM_GA_file)
RCP45_DYN_EnergyB_MAM_GA_file.close()

RCP45_DYN_EnergyB_DJF_GA_file = open("RCP45_DYN_EnergyB_DJF_GA.pickle","wb")
pickle.dump(RCP45_DYN_EnergyB_DJF_GA,RCP45_DYN_EnergyB_DJF_GA_file)
RCP45_DYN_EnergyB_DJF_GA_file.close()